In [455]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import multivariate_normal
from sklearn.cluster import KMeans

# 1.2 Experiment (20 Points)

In [468]:
def k_means(data, k, max_iter = 100, n_restarts = 100):
    np.random.seed(0)
    objective_min = np.inf
    centroid_best = None
    labels_best = None
    for restart in range(n_restarts):
        centroids = data[np.random.choice(data.shape[0], k, replace = False), :]
        for i in range(max_iter):
            #data.shape = (n_sample, n_features) = (1500, 2)
            #centroids.shape = (n_clusters, n_features) = (5, 2)
            # (n_clusters, n_samples, n_features) = (5, 1, 2)
            # data - centroids[:, np.newaxis] = (n_clusters, n_samples, n_features)
            # sum(axis = 2) result resulting in an array of shape (n_clusters, n_samples) containing the squared 
            #    Euclidean distances between each data point and each centroid.
            labels = np.argmin(((data - centroids[:, np.newaxis])**2).sum(axis = 2), axis = 0)
            # returns the indices of the minimum values along columns.
            for j in range(k):
                # choose columns with a row value of true
                # in this case, there're 5 centroids
                centroids[j] = data[labels == j, :].mean(axis = 0)
        # picking the one with the lowest objective value
        objective = np.sum(((data - centroids[labels, :])**2).sum(axis = 1))
        if objective_min > objective:
            objective_min = objective
            centroid_best = centroids
            labels_best = labels

    return centroid_best, labels_best

In [469]:
# Use k-means++ initialization strategy
# k-means++ algorithm selects the initial centroids to be well-spaced and tries to improve the convergence of the k-means algorithm.

def kmean_plus(data, k, max_iter = 100, n_restarts = 1000):
    """
    kmeans.labels_ : returns an array of cluster labels, where each element corresponds 
        to a data point and specifies which cluster it belongs to.
    centroids[labels] uses these labels to index the centroids array, so that 
        each data point is paired with its assigned centroid.
    """
    np.random.seed(0)
    objective_min = np.inf
    centroid_best = None

    for restart in range(n_restarts):
        kmeans = KMeans(n_clusters= k, init = 'k-means++', max_iter = max_iter).fit(data)
        labels = kmeans.labels_
        centroids = kmeans.cluster_centers_
        # calculates the sum of squared distances between each data point and its assigned centroid.
        objective = ((data - centroids[labels])).sum()

        if objective_min > objective:
            objective_min = objective
            centroid_best = centroids
            labels_best = labels
        
    return centroid_best, labels_best

#### Expectation maximization algorith for GMMs:

In [470]:
# responsibility matrix for each sample and cluster
def compute_respon(data, k, means, covariances, weights):
    n_data, n_features = data.shape
    respon = np.zeros((n_data, k))
    for j in range(k):
        # calculates the probability density function (PDF) of a multivariate normal distribution 
        # at a given set of points.
        # used in the E-step to compute the responsibilities of each data point for each mixture component.
        # calculating the posterior probability of each data point being in the j-th cluster. 
        respon[:, j] = weights[j] * multivariate_normal.pdf(data, mean = means[j], cov = covariances[j])
    # respon has shape (n_data, k), normal has shape (n_data, 1)
    # Therefore, we need to add extra axis so that it has the same shape as respon
    normal = np.sum(respon, axis = 1)[:, np.newaxis]
    #normalize the responsibility across each cluster
    return respon/normal

In [471]:
def gmm(data, k, max_iter = 100, n_restarts = 10):
    np.random.seed(0)
    obj_min = np.inf
    best_means = None
    best_covs = None
    best_weights = None
    best_labels = None
    n_data, n_features = data.shape

    for ind in range(n_restarts):
        # Initialize parameters
        # the weights are initialized uniformly.
        # the means are randomly chosen from the input data.
        # the covariance matrices are initialized as the identity matrix.
        weights = np.ones(k) / k
        means = data[np.random.choice(n_data, k, replace = False), :]
        cov = np.array([np.eye(n_features) for i in range (k)])
        # E-step: compute responsibilities
        for i in range(max_iter):
            likelihoods = np.array([multivariate_normal.pdf(data, means[j], cov[j], allow_singular= True) for j in range(k)]).T 
            # weights = (k,)
            # (n_data, k): The probability of each data belong to one cluster
            joint_prob = likelihoods * weights
            respon = joint_prob / joint_prob.sum(axis = 1)[:, np.newaxis, :]
            #respon = compute_respon(data, k, means, cov, weights)
            # Nk[i] represents the sum of responsibilities for cluster 'i' over all data points. 
            # This value is used in the M-step of the EM algorithm to update the 
            # cluster weights, means, and covariances.
            Nk = np.sum(respon, axis = 0)
            # ensures that the weights sum up to 1, 
            # which is a requirement for probabilities.
            # Updates the prior probabilities of the data points
            # belonging to each of the Gaussian mixture components.
            weights = Nk/n_data
            # updated based on the current responsibilities and the data.
            # data = (1500, 2) = (n_data, n_features)
            # respon = (1500, 5) = (n_data, k)'
            # Weighted sum of the data points, where the weights are given by
            # the responsibilities of the Gaussian components for each data point.
            # Nk[:, np.newaxis] has shape (k, 1)
            # divide each row of the weighted sum by the corresponding value of Nk
            means = (respon.T @ data) / Nk[: , np.newaxis]
            # means = (k,n_features)
            for j in range(k):
                # (num_data, num_features)
                diff = data - means[j]
                cov[j] = (respon[:, j, np.newaxis]).T @ diff / Nk[j]
        objective = -np.log(likelihoods.sum(axis = 1)).sum()
        if objective < obj_min:
            obj_min = objective
            best_covs = cov
            best_means = means
            best_weights = weights
    labels = np.argmax(respon, axis = 1)
    return best_means, best_covs, best_weights, obj_min

In [472]:
np.ones(5).shape

(5,)

In [473]:
np.array([np.eye(2) for i in range (5)])

array([[[1., 0.],
        [0., 1.]],

       [[1., 0.],
        [0., 1.]],

       [[1., 0.],
        [0., 1.]],

       [[1., 0.],
        [0., 1.]],

       [[1., 0.],
        [0., 1.]]])

## Clustering accuracy

In [ ]:
sigmas = [0.5, 1, 2, 4, 8]
mu1 = np.array([-1, -1])
mu2 = np.array([1, -1])
mu3 = np.array([0, 1])
sigma1 = np.array([[2, 0.5], [0.5, 1]])
sigma2 = np.array([[1, -0.5], [-0.5, 2]])
sigma3 = np.array([[1, 0], [0, 2]])
np.random.seed(0)
numpoints = 100
data = np.empty((0, 2), float)
true_label = []
for sigma in sigmas:
    multi = np.random.multivariate_normal(mu1, sigma*sigma1, numpoints)
    multi2 = np.random.multivariate_normal(mu2, sigma*sigma2, numpoints)
    multi3 = np.random.multivariate_normal(mu3, sigma*sigma3, numpoints)
    dataset = np.concatenate((multi, multi2, multi3))
    true_label.extend([0] * 100)
    true_label.extend([1] * 100)
    true_label.extend([2] * 100)
    data = np.concatenate((data, dataset)) 

In [481]:
def compute_cluster_accuracy(means, true_labels):
    labels_pred = np.zeros_like(true_labels)
    for i in range (len(means)):
        dist = np.sqrt(((means[i] - [mu1, mu2, mu3])**2).sum(axis = 1))
        labels_pred[labels_pred == i] = np.argmin(dist) #find the min mu from the 3 mu
    return np.mean(labels_pred == true_labels)


In [482]:
def kmean_cluster_accuracy(centroids, labels, true_labels):
    """
    labels: an array containing index of  cluster produced by one of the above 
    true_labels: array containing the true cluster assignments 
    """
    true_means = np.array([[-1, -1], [1, -1], [0, 1]])
    mapping = []
    for i in range(3):
        idx = (labels == i)
        idx = np.nonzero(idx)[0] # get the index 
        # if a cluster is empty, it is assigned a mapping of -1 
        if np.sum(idx) == 0:
            mapping.append(-1)
            # to skip the current iteration of the loop and move on to the next iteration.
            continue
        distances = np.linalg.norm(centroids[i] - true_means, axis = 1)
        closest_means = np.argmin(distances) #location of the closest true mean
        mapping.append(closest_means)
    predicted_labels = np.array([mapping[label] for label in labels])
    accuracy = (predicted_labels == true_labels).mean()
    return accuracy

In [483]:
centroid_best, labels_best = k_means(data, 3)


In [484]:
kmean_cluster_accuracy(centroid_best, labels_best, true_label)

0.642

In [485]:
centroid_best_plus, labels_best_plus = kmean_plus(data, 3)

In [486]:
kmean_cluster_accuracy(centroid_best_plus, labels_best_plus, true_label)

0.6433333333333333